In [22]:
import configparser

def get_key(app):
    config = configparser.ConfigParser()
    config.read('../../key/config.ini')
    api_key = config.get(app, 'api_key')
    return api_key

def get_gaode_key():
    return get_key("GaoDe")

def get_openai_key():
    return get_key("OpenAI")

In [23]:
import openai

openai.api_key = get_openai_key()#填充自己的key

def get_completion(prompt, model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]


In [24]:
import requests

text_mrqx = requests.get("https://raw.githubusercontent.com/hankinghu/literature-books/master/命若琴弦.txt").text

prompt =f'''请简要以下小说

{text_mrqx[235:-145]}
'''

print(len(text_mrqx))

12627


In [112]:
print(get_completion(prompt,model="gpt-3.5-turbo"))

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 16135 tokens. Please reduce the length of the messages.

In [25]:
print(get_completion(prompt,model="gpt-3.5-turbo-16k"))

这个故事讲述了两个瞎子，一老一少，他们在偏僻的群山中走动，以说书为生。老瞎子希望能弹断一千根琴弦，以实现他的愿望。他们来到野羊坳村，小瞎子遇到了一个叫兰秀儿的女孩，他们之间产生了感情。然而，老瞎子意识到这段感情可能会给他们带来麻烦，他决定离开野羊坳。在离开之前，他弹断了最后几根琴弦。然后，老瞎子离开了，留下了小瞎子和兰秀儿。小瞎子病倒了，但他仍然等待着老瞎子的回


In [135]:
import openai
import json
import requests
openai.api_key = get_openai_key()


#调用GPT模型
def get_completion_from_messages(messages,functions, model="gpt-3.5-turbo-0613", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        functions=functions
    )
    return response.choices[0]


def get_current_weather(para):
    city = para["city"]
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?parameters'
    params = {
        'key': get_gaode_key(),
        'city': city,
        'extensions': 'base',
        'output': 'JSON',
        'extensions':'all'
        
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    return data['forecasts'][0]['casts'][0]

def send_email(para):
    addr = para["addr"]
    title = para["title"]
    content = para["content"]
    text = f"To {addr}\n{title}\n{content}"
    #print("邮件已经发送：\n "+text)
    return {"status":"邮件已经发送","email":text}

functions = [
    {
        "name": "get_current_weather",
        "description": "获取一个城市的当前天气",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名称"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "send_email",
        "description": "给指定地址发送电子邮件",
        "parameters": {
            "type": "object",
            "properties": {
                "addr": {
                    "type": "string",
                    "description": "电子邮件地址"
                },
                "title": {
                    "type": "string",
                    "description": "邮件标题"
                },
                "content": {
                    "type": "string",
                    "description": "邮件内容"
                }
            },
            "required": ["addr","title","content"]
        }
    }
]


#向GPT提问
def get_completion(prompt,functions):
    #拼接消息
    messages = [{"role": "user", "content": prompt}]
    
    #第一轮调用GPT
    response = get_completion_from_messages(messages,functions)
    
    #如果返回终止原因是API调用，本地执行
    if response['finish_reason'] == "function_call":   
        arguments = response["message"]["function_call"]["arguments"]
        function =  response["message"]["function_call"]["name"]

        result = eval(f"{function}({arguments})")#根据API名称和参数动态调用本地接口
        
        #把GPT的调用指令和API返回结果再次发送给GPT
        messages.append(response["message"])
        messages.append( {"role": "function", "name": "get_current_weather", "content":str(result)})
        response = get_completion_from_messages(messages,functions)
        
    print(response["message"]["content"])
    

get_completion("我在杭州,今天出门要带伞吗",functions)

In [133]:
get_completion("我在杭州,今天出门要带伞吗",functions)

根据当前天气情况，杭州今天有小雨，建议您出门时带上一把伞。


In [136]:
get_completion("帮我发一封市场营销部门周会邮件，内容包含周会时间、地点、主题，周会时间是下周二14点，地点在蓝色宇宙会议室，主题是618大促简要复盘，部门邮件地址是runing@spaceK.com",functions)

邮件已经发送至 runing@spaceK.com，邮件标题为 "市场营销部门周会通知"，邮件内容如下：

尊敬的市场营销部门成员，

本周会将于下周二14点在蓝色宇宙会议室举行。

会议主题是618大促简要复盘，请大家准时参加。

谢谢！

市场营销部门


In [137]:
get_completion("我在长沙,今天出门要带伞吗",functions)

根据当前天气情况，长沙今天有小雨，建议您出门时带上一把伞。


In [138]:
get_completion("我在成都,今天出门要带伞吗",functions)

根据成都的天气预报，今天是多云的天气，白天最高温度为28℃，夜间最低温度为21℃。不需要带伞。


In [130]:

openai.api_key = get_openai_key()


import requests
def get_current_weather(para):
    #获取当天的天气预报
    city = para["city"]
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?parameters'
    params = {
        'key': get_gaode_key(),#填入自己的高德API
        'city': city,
        'extensions': 'base',
        'output': 'JSON',
        'extensions':'all'
        
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    return data['forecasts'][0]['casts'][0]

functions = [
    {
        "name": "get_current_weather",
        "description": "获取一个城市的当前天气",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名称"
                }
            },
            "required": ["city"]
        }
    },
]


import openai
import json

#调用GPT模型
def get_completion_from_messages(messages,functions, model="gpt-3.5-turbo-0613", temperature=0):
    print("-----------------------------------------\n输入messages:"+str(json.dumps(messages, indent=1, ensure_ascii=False)))
    print("输入functions:"+str(json.dumps(functions, indent=1, ensure_ascii=False)))
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        functions=functions
    )
    
    print("输出:"+json.dumps(response, indent=1, ensure_ascii=False))
    return response.choices[0]



#向GPT提问
def ask_gpt(prompt,functions):
    #拼接消息
    messages = [{"role": "user", "content": prompt}]
    
    #第一轮调用GPT
    response = get_completion_from_messages(messages,functions)
    
    #如果返回终止原因是API调用，本地执行
    if response['finish_reason'] == "function_call":   
        arguments = response["message"]["function_call"]["arguments"]
        function =  response["message"]["function_call"]["name"]

        result = eval(f"{function}({arguments})")#根据API名称和参数动态调用本地接口
        
        #把GPT的调用指令和API返回结果再次发送给GPT
        messages.append(response["message"])
        messages.append( {"role": "function", "name": "get_current_weather", "content":str(result)})
        response = get_completion_from_messages(messages,functions)
        
    print(response["message"]["content"])

get_completion("我在成都,今天出门要带伞吗",functions)
    

-----------------------------------------
输入messages:[
 {
  "role": "user",
  "content": "我在成都,今天出门要带伞吗"
 }
]
输入functions:[
 {
  "name": "get_current_weather",
  "description": "获取一个城市的当前天气",
  "parameters": {
   "type": "object",
   "properties": {
    "city": {
     "type": "string",
     "description": "城市名称"
    }
   },
   "required": [
    "city"
   ]
  }
 }
]
输出:{
 "id": "chatcmpl-7SPqEy1Iz16iVwHVpE2jwpECvaehw",
 "object": "chat.completion",
 "created": 1687006874,
 "model": "gpt-3.5-turbo-0613",
 "choices": [
  {
   "index": 0,
   "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"city\": \"成都\"\n}"
    }
   },
   "finish_reason": "function_call"
  }
 ],
 "usage": {
  "prompt_tokens": 73,
  "completion_tokens": 17,
  "total_tokens": 90
 }
}
-----------------------------------------
输入messages:[
 {
  "role": "user",
  "content": "我在成都,今天出门要带伞吗"
 },
 {
  "role": "assistant",
  "content": n

In [124]:
print(get_current_weather({"city":"北京"}))

{'date': '2023-06-17', 'week': '6', 'dayweather': '多云', 'nightweather': '晴', 'daytemp': '38', 'nighttemp': '22', 'daywind': '东南', 'nightwind': '东南', 'daypower': '≤3', 'nightpower': '≤3', 'daytemp_float': '38.0', 'nighttemp_float': '22.0'}
